In [2]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import tomllib
from pathlib import Path
from pandas import DataFrame
from common.utils import load_dataset, optimize_memory, get_params, DatasetType
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import joblib
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import xgboost as xgb
import optuna
import json
import pickle

In [3]:
color = sns.color_palette()

# Google Brain - Ventilator Pressure Prediction

<br />
 
##### Overview

---
<div><p>What do doctors do when a patient has trouble breathing? They use a ventilator to pump oxygen into a sedated patient's lungs via a tube in the windpipe. But mechanical ventilation is a clinician-intensive procedure, a limitation that was prominently on display during the early days of the COVID-19 pandemic. At the same time, developing new methods for controlling mechanical ventilators is prohibitively expensive, even before reaching clinical trials. High-quality simulators could reduce this barrier. </p>
<p>Current simulators are trained as an ensemble, where each model simulates a single lung setting. However, lungs and their attributes form a continuous space, so a parametric approach must be explored that would consider the differences in patient lungs. </p>
<p>Partnering with Princeton University, the team at Google Brain aims to grow the community around machine learning for mechanical ventilation control. They believe that neural networks and deep learning can better generalize across lungs with varying characteristics than the current industry standard of PID controllers.  </p>
<p>In this competition, you’ll simulate a ventilator connected to a sedated patient's lung. The best submissions will take lung attributes compliance and resistance into account.</p>
<p>If successful, you'll help overcome the cost barrier of developing new methods for controlling mechanical ventilators. This will pave the way for algorithms that adapt to patients and reduce the burden on clinicians during these novel times and beyond. As a result, ventilator treatments may become more widely available to help patients breathe.</p>

##### Evaluation

---

The competition will be scored as the mean absolute error between the predicted and actual pressures during the inspiratory phase of each breath. The expiratory phase is not scored. The score is given by:

|X−Y|

where X
is the vector of predicted pressure and Y

is the vector of actual pressures across all breaths in the test set.
Submission File

For each id in the test set, you must predict a value for the pressure variable. The file should contain a header and have the following format:

<br/>

##### Submission File

----

For each id in the test set, you must predict a value for the pressure variable. The file should contain a header and have the following format:

```csv
id,pressure
1,20
2,23
3,24
etc.
```

##### Dataset Description

---

The ventilator data used in this competition was produced using a modified open-source ventilator connected to an artificial bellows test lung via a respiratory circuit. The diagram below illustrates the setup, with the two control inputs highlighted in green and the state variable (airway pressure) to predict in blue. The first control input is a continuous variable from 0 to 100 representing the percentage the inspiratory solenoid valve is open to let air into the lung (i.e., 0 is completely closed and no air is let in and 100 is completely open). The second control input is a binary variable representing whether the exploratory valve is open (1) or closed (0) to let air out.

In this competition, participants are given numerous time series of breaths and will learn to predict the airway pressure in the respiratory circuit during the breath, given the time series of control inputs.

![image](../docs/ventilator-diagram.png)

Each time series represents an approximately 3-second breath. The files are organized such that each row is a time step in a breath and gives the two control signals, the resulting airway pressure, and relevant attributes of the lung, described below.

<br>

<table>
  <thead>
    <tr>
      <th>File</th>
      <th>Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>train.csv</td>
      <td>The training set</td>
    </tr>
    <tr>
      <td>test.csv</td>
      <td>The test set</td>
    </tr>
    <tr>
      <td>sample_submission.csv</td>
      <td>A sample submission file in the correct format</td>
    </tr>
  </tbody>
</table>

<br>

<table>
  <thead>
    <tr>
      <th>Column</th>
      <th>Description</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>id</td>
      <td>Globally-unique time step identifier across an entire file</td>
    </tr>
    <tr>
      <td>breath_id</td>
      <td>Globally-unique time step for breaths</td>
    </tr>
    <tr>
      <td>R</td>
      <td>Lung attribute indicating how restricted the airway is (in cmH2O/L/S). This is the change in pressure per change in flow.</td>
    </tr>
    <tr>
      <td>C</td>
      <td>Lung attribute indicating how compliant the lung is (in mL/cmH2O). This is the change in volume per change in pressure.</td>
    </tr>
    <tr>
      <td>time_step</td>
      <td>The actual time stamp</td>
    </tr>
    <tr>
      <td>u_in</td>
      <td>The control input for the inspiratory solenoid valve. Ranges from 0 to 100.</td>
    </tr>
    <tr>
      <td>u_out</td>
      <td>The control input for the exploratory solenoid valve. Either 0 or 1.</td>
    </tr>
    <tr>
      <td>pressure</td>
      <td>The airway pressure measured in the respiratory circuit, measured in cmH2O.</td>
    </tr>
  </tbody>
</table>

In [4]:
def objective(trial: optuna.trial.Trial):
    train_df: DataFrame = load_dataset("ventilator-pressure-prediction", DatasetType.TRAIN,
                                       index=False)

    # train_df, _ = optimize_memory(train_df)

    X_train, X_test, y_train, y_test = train_test_split(
        train_df.drop('pressure', axis=1),
        train_df['pressure'],
        test_size=0.2,
        random_state=42)

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 100, step=5),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.05, step=0.04),
        'max_depth': trial.suggest_int('max_depth', 4, 6, step=2),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0, step=0.5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.5),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 100.0),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 100.0),
        'random_state': 42,
        'verbosity': 3
    }

    model = XGBRegressor(**params)

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_pred)

    return rmse


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

print("Best hyperparameters: ", study.best_params)
print("Best RMSE: ", study.best_value)

[I 2025-04-27 18:35:45,166] A new study created in memory with name: no-name-37d3a2e0-91f3-4df7-b3fe-02c15faf66e3


[18:35:49] ======== Monitor (0): HostSketchContainer ========
[18:35:49] AllReduce: 0.003603s, 1 calls @ 3603us

[18:35:49] MakeCuts: 0.003645s, 1 calls @ 3645us

[18:35:49] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[18:36:00] ======== Monitor (0): Learner ========
[18:36:00] Configure: 0.00032s, 1 calls @ 320us

[18:36:00] EvalOneIter: 0.001046s, 85 calls @ 1046us

[18:36:00] GetGradient: 0.30635s, 85 calls @ 306350us

[18:36:00] PredictRaw: 0.005236s, 85 calls @ 5236us

[18:36:00] UpdateOneIter: 11.0369s, 85 calls @ 11036882us

[18:36:00] ======== Monitor (0): GBTree ========
[18:36:00] BoostNewTrees: 10.6851s, 85 calls @ 10685088us

[18:36:00] CommitModel: 0.000139s, 85 calls @ 139us

[18:36:00] ======== Monitor (0): HistUpdater ========
[18:36:00] BuildHistogram: 2.87302s, 425 calls @ 2873018us

[18:36:00] EvaluateSplits: 0.118509s, 510 calls @ 118509us

[18:3

[I 2025-04-27 18:36:00,796] Trial 0 finished with value: 20.134315177338305 and parameters: {'n_estimators': 85, 'learning_rate': 0.05, 'max_depth': 6, 'subsample': 1.0, 'colsample_bytree': 0.5, 'reg_alpha': 77.48101027479223, 'reg_lambda': 96.42436791051622}. Best is trial 0 with value: 20.134315177338305.


[18:36:04] ======== Monitor (0): HostSketchContainer ========
[18:36:04] AllReduce: 0.003629s, 1 calls @ 3629us

[18:36:04] MakeCuts: 0.003663s, 1 calls @ 3663us

[18:36:04] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[18:36:10] ======== Monitor (0): Learner ========
[18:36:10] Configure: 0.000308s, 1 calls @ 308us

[18:36:10] EvalOneIter: 0.000641s, 50 calls @ 641us

[18:36:10] GetGradient: 0.145018s, 50 calls @ 145018us

[18:36:10] PredictRaw: 0.005503s, 50 calls @ 5503us

[18:36:10] UpdateOneIter: 6.02019s, 50 calls @ 6020189us

[18:36:10] ======== Monitor (0): GBTree ========
[18:36:10] BoostNewTrees: 5.82867s, 50 calls @ 5828674us

[18:36:10] CommitModel: 7e-05s, 50 calls @ 70us

[18:36:10] ======== Monitor (0): HistUpdater ========
[18:36:10] BuildHistogram: 1.16182s, 250 calls @ 1161820us

[18:36:10] EvaluateSplits: 0.011381s, 300 calls @ 11381us

[18:36:10] 

[I 2025-04-27 18:36:10,902] Trial 1 finished with value: 22.788070433647643 and parameters: {'n_estimators': 50, 'learning_rate': 0.05, 'max_depth': 6, 'subsample': 0.5, 'colsample_bytree': 0.5, 'reg_alpha': 18.630520129838096, 'reg_lambda': 10.531979384007585}. Best is trial 0 with value: 20.134315177338305.


...

[18:37:20] ======== Monitor (0): HostSketchContainer ========
[18:37:20] AllReduce: 0.003626s, 1 calls @ 3626us

[18:37:20] MakeCuts: 0.003656s, 1 calls @ 3656us

[18:37:20] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[18:37:26] ======== Monitor (0): Learner ========
[18:37:26] Configure: 0.000262s, 1 calls @ 262us

[18:37:26] EvalOneIter: 0.000834s, 75 calls @ 834us

[18:37:26] GetGradient: 0.157031s, 75 calls @ 157031us

[18:37:26] PredictRaw: 0.005214s, 75 calls @ 5214us

[18:37:26] UpdateOneIter: 5.77854s, 75 calls @ 5778537us

[18:37:26] ======== Monitor (0): GBTree ========
[18:37:26] BoostNewTrees: 5.57526s, 75 calls @ 5575259us

[18:37:26] CommitModel: 8.5e-05s, 75 calls @ 85us

[18:37:26] ======== Monitor (0): HistUpdater ========
[18:37:26] BuildHistogram: 1.43044s, 375 calls @ 1430438us

[18:37:26] EvaluateSplits: 0.01558s, 450 calls @ 15580us

[18:37:26]

[I 2025-04-27 18:37:26,336] Trial 8 finished with value: 17.837457264722598 and parameters: {'n_estimators': 75, 'learning_rate': 0.05, 'max_depth': 6, 'subsample': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 53.238332279752974, 'reg_lambda': 56.86919241529784}. Best is trial 8 with value: 17.837457264722598.


[18:37:30] ======== Monitor (0): HostSketchContainer ========
[18:37:30] AllReduce: 0.003621s, 1 calls @ 3621us

[18:37:30] MakeCuts: 0.003689s, 1 calls @ 3689us

[18:37:30] DEBUG: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\gbm\gbtree.cc:130: Using tree method: 0
[18:37:37] ======== Monitor (0): Learner ========
[18:37:37] Configure: 0.000237s, 1 calls @ 237us

[18:37:37] EvalOneIter: 0.000837s, 65 calls @ 837us

[18:37:37] GetGradient: 0.182606s, 65 calls @ 182606us

[18:37:37] PredictRaw: 0.004657s, 65 calls @ 4657us

[18:37:37] UpdateOneIter: 7.06811s, 65 calls @ 7068111us

[18:37:37] ======== Monitor (0): GBTree ========
[18:37:37] BoostNewTrees: 6.84339s, 65 calls @ 6843391us

[18:37:37] CommitModel: 9.5e-05s, 65 calls @ 95us

[18:37:37] ======== Monitor (0): HistUpdater ========
[18:37:37] BuildHistogram: 1.95862s, 325 calls @ 1958624us

[18:37:37] EvaluateSplits: 0.033114s, 390 calls @ 33114us

[18:37:37

[I 2025-04-27 18:37:37,420] Trial 9 finished with value: 18.094136996565254 and parameters: {'n_estimators': 65, 'learning_rate': 0.05, 'max_depth': 6, 'subsample': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 17.295529669905072, 'reg_lambda': 26.98740330095737}. Best is trial 8 with value: 17.837457264722598.


Best hyperparameters:  {'n_estimators': 75, 'learning_rate': 0.05, 'max_depth': 6, 'subsample': 1.0, 'colsample_bytree': 1.0, 'reg_alpha': 53.238332279752974, 'reg_lambda': 56.86919241529784}
Best RMSE:  17.837457264722598


In [8]:
best_params = study.best_trial.params
best_params.update({
    'verbosity': 3,
})

with open("../models/ventilator_pressure-prediction_model.pkl", "wb") as f:
    pickle.dump(best_params, f)

In [12]:
model: XGBRegressor = XGBRegressor()

with open("../models/ventilator_pressure-prediction_model.pkl", "rb") as f:
    params = pickle.load(f)
    model = XGBRegressor(**params)

model.get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 1.0,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.05,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 6,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': 75,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': 53.238332279752974,
 'reg_lambda': 56.86919241529784,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 1.0,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': 3}